# AN2DL - Second Challenge

## Initial Operations

### Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers

### Set Seed

In [2]:
seed = 69
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Load the dataset

In [3]:
dataset = pd.read_csv('../Data/training.csv')
print(dataset.shape)
dataset.head()

(68528, 7)


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,7.97698,4.33494,10.67282,1.76692,3.22440,51.681460,3.65434
1,8.07824,4.44616,10.56160,1.70716,3.32566,51.563598,3.47672
2,8.02844,4.22372,10.56160,1.64906,3.17460,50.863080,3.47672
3,8.02844,4.22372,10.56160,1.70716,3.17460,45.841581,3.47672
4,7.87572,4.44616,10.45038,1.70716,3.27586,47.126421,3.47672


### Train-Test Split and Normalization

In [4]:
X_train_raw = dataset.iloc[:-1000]
X_test = dataset.iloc[-1000:]

# Normalize train samples
X_min = X_train_raw.min()
X_max = X_train_raw.max()
X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

In [6]:
print(X_train_raw.shape) 
print(X_test.shape)

(67528, 7)
(1000, 7)


## Model

### Utility Function to Define Sequences

In [ ]:
def build_sequences(df, target_labels, window, stride, telescope):
    
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

### Utility Function to Define Model Based on Hyperparameters

In [ ]:
def build_model(input_shape, output_shape):
    
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    layer = tfkl.Bidirectional(tfkl.GRU(64, return_sequences=True, name='GRU1'),name='Bidirectional1')(input_layer)
    layer = tfkl.Conv1D(64, 3, padding='same', activation='relu', name='Convolution1')(layer)
    layer = tfkl.MaxPool1D(name='MaxPooling1')(layer)
    layer = tfkl.Bidirectional(tfkl.GRU(128, return_sequences=True, name='GRU2'), name='Bidirectional2')(layer)
    layer = tfkl.Conv1D(128, 6, padding='same', activation='relu', name='Convolutional2')(layer)
    layer = tfkl.MaxPool1D(name='MaxPooling2')(layer)
    layer = tfkl.Bidirectional(tfkl.GRU(256, return_sequences=True, name='GRU3'), name='Bidirectional3')(layer)
    layer = tfkl.Conv1D(256, 12, padding='same', activation='relu', name='Convolutional3')(layer)
    layer = tfkl.GlobalAveragePooling1D(name='GlobalAveragePooling')(layer)
    layer = tfkl.Dropout(.25, name='Dropout')(layer)
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', name='Dense')(layer)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]), name='Reshape')(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same', name='Output')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model

### Utility Function to Predict the Future

In [ ]:
def predict_future(df,window,telescope,pred_horizon):
    
  # Autoregressive Forecasting
  X = tf.constant(df)
  X = X.numpy()
  future = X[-window:]
  X_min = X.min(axis=0)
  X_max = X.max(axis=0)
  future = (future-X_min)/(X_max-X_min)
  future = np.expand_dims(future,0)
  out = np.array([])
  for i in range(0, pred_horizon, telescope):
      current_prediction = model.predict(future)
      if(len(out) == 0):
          out = current_prediction
      else:
          out = np.concatenate((out,current_prediction),axis=1)
      future = np.concatenate((future[:,telescope:,:],current_prediction), axis=1)
  out = np.squeeze(out,0)
  out = out * (X_max - X_min) + X_min
  out = out[:pred_horizon]
  return out

### Hyperparameters Search

In [ ]:
W = [50,100,200,300,400,500]
S = [5,10,25]
T = [27,36,48,54,108,216,432]

target_labels = dataset.columns
rmse_list = []
for i in range(20):
    print("Iteration: ",i)
    
    # Build the training data
    window = random.choice(W)
    stride = int(window / random.choice(S))
    telescope = random.choice(T)
    X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
    input_shape = X_train.shape[1:]
    output_shape = y_train.shape[1:]
    print("Parameters: ",[window, stride, telescope])
    
    # Build the model
    model = build_model(input_shape, output_shape)
    
    # Train the model
    batch_size = 64
    epochs = 20
    model.fit(
      x = X_train,
      y = y_train,
      batch_size = batch_size,
      epochs = epochs
    )
    
    # Evaluate performance on test set
    out = predict_future(dataset.iloc[:-1000],window,telescope,1000)
    rmse = mean_squared_error(tf.constant(X_test).numpy(), out, multioutput='uniform_average', squared=True)
    rmse_list.append(rmse)
    print("RMSE: ",rmse)

In [ ]:
val, idx = min((val, idx) for (idx, val) in enumerate(rmse_list))
print(idx,val)